<a href="https://colab.research.google.com/github/rudevico/Gachon-AISTUDY/blob/main/Diabetes_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0. Introduction

[Download Sample data set: diabetes.csv](https://github.com/plotly/datasets/blob/master/diabetes.csv)  
Diabetes(당뇨병) data set에 대해서 여러 가지의 Clssification algorithm을 적용해서 분류해보겠다.  

**types of Classification algorithms that we use**  
1. K-Nearest Neighbors; K-NN
2. Logistic Regression; LR
3. Decision Tree; DT
4. Random Forest; RF
5. Support Vector Machine; SVM

## 0. 1. Load data set

In [1]:
import pandas as pd

diabetes = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/diabetes.csv')
diabetes.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
diabetes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


In [3]:
diabetes.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


In [4]:
pd.unique(diabetes['Outcome'])

array([1, 0])

In [5]:
pd.value_counts(diabetes['Outcome'])

Outcome
0    500
1    268
Name: count, dtype: int64

## 0. 2. Description of data set

**8 Features**  
  * $x_1$, Pregnancies(임신 횟수)
  * $x_2$, Glucose
  * $x_3$, BloodPressure
  * $x_4$, SkinThickness(삼두근 피부 두께)
  * $x_5$, Insulin
  * $x_6$, BMI
  * $x_7$, DiabetesPedigreeFunction(당뇨 유전병 가중치)
  * $x_8$, Age

**Label**  
  * $y$, Outcome
    - 500 negative class, 268 positive class



## 0. 3. Splitting the data set into X(features) and y(label)

In [6]:
X = diabetes.drop('Outcome', axis=1)
y = diabetes['Outcome']
print(X[:3])

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6      148             72             35        0  33.6   
1            1       85             66             29        0  26.6   
2            8      183             64              0        0  23.3   

   DiabetesPedigreeFunction  Age  
0                     0.627   50  
1                     0.351   31  
2                     0.672   32  


## 0. 4. Convert pandas dataframe to numpy array

In [7]:
X = X.to_numpy()
y = y.to_numpy()

## 0. 5. Splitting the Data sets(X, y) into train set and test set
> `stratify` parameter를 `y`로 지정한 이유에 대해서는 **0. 5. 1.**을 참고

In [8]:
from sklearn.model_selection import train_test_split

# default, test_size=0.25, shuffle=True, stratify=None
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, shuffle=True, stratify=y, random_state=42)
print(X_train.shape, X_test.shape)

(614, 8) (154, 8)


### 0. 5. 1. Why we need the `shuffle` and `stratify` parameters?

본 예제의 data set이 아닌, **별도의 data set을 사용**해서 설명하도록 하겠다.

In [9]:
# parametet 설명을 위한 별도의 data set
bream_length = [25.4, 26.3, 26.5, 29.0, 29.0, 29.7, 29.7, 30.0, 30.0, 30.7, 31.0, 31.0,
                31.5, 32.0, 32.0, 32.0, 33.0, 33.0, 33.5, 33.5, 34.0, 34.0, 34.5, 35.0,
                35.0, 35.0, 35.0, 36.0, 36.0, 37.0, 38.5, 38.5, 39.5, 41.0, 41.0]
bream_weight = [242.0, 290.0, 340.0, 363.0, 430.0, 450.0, 500.0, 390.0, 450.0, 500.0, 475.0, 500.0,
                500.0, 340.0, 600.0, 600.0, 700.0, 700.0, 610.0, 650.0, 575.0, 685.0, 620.0, 680.0,
                700.0, 725.0, 720.0, 714.0, 850.0, 1000.0, 920.0, 955.0, 925.0, 975.0, 950.0]

smelt_length = [9.8, 10.5, 10.6, 11.0, 11.2, 11.3, 11.8, 11.8, 12.0, 12.2, 12.4, 13.0, 14.3, 15.0]
smelt_weight = [6.7, 7.5, 7.0, 9.7, 9.8, 8.7, 10.0, 9.9, 9.8, 12.2, 13.4, 12.2, 19.7, 19.9]

fish_length = bream_length + smelt_length
fish_weight = bream_weight + smelt_weight

tX = [ [l, w] for l, w in zip(fish_length, fish_weight) ]
ty = [1] * 35 + [0] * 14

In [10]:
print(ty)

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


(위 cell)  
해당 data set은 sample의 수가 총 39개이고, 먼저 positive class가 모두 나열된 후에 negative class가 나열되는 특징을 갖는다.  
또한 The ratio between the positive class and the negative class is $ 35 : 14 $.

* * *
먼저, `shuffle` parameter에 대해서 알아보자.

In [11]:
# ex1-1. shuffle=False, stratify=None(default)
tX_train, tX_test, ty_train, ty_test = train_test_split(
    tX, ty, test_size=0.2, shuffle=False, random_state=42)

In [12]:
print(ty_train)
print(ty_train.count(1), ty_train.count(0))
print(ty_test)
print(ty_test.count(1), ty_test.count(0))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0]
35 4
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
0 10


(위 cell)  
`shuffle=False`인 경우에는 data set의 처음부터 순서대로 $49*0.8$개 만큼을 split하여 tX_train으로 사용하고, 이후부터 순서대로 $49*0.2$개 만큼을 tX_test로 사용한다.  
따라서 다음과 같은 문제가 발생한다.  
* 4개의 negative class에 대해서만 training됨
* positive class에 대한 test는 단 1개도 할 수 없음

In [13]:
# ex1-2. shuffle=True(default), stratify=None(default)
tX_train, tX_test, ty_train, ty_test = train_test_split(
    tX, ty, test_size=0.2, shuffle=True, random_state=19)

In [14]:
print(ty_train)
print(ty_train.count(1), ty_train.count(0))
print(ty_test)
print(ty_test.count(1), ty_test.count(0))

[1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1]
30 9
[0, 0, 0, 1, 1, 1, 0, 0, 1, 1]
5 5


(위 cell)  
`shuffle=True`인 경우에는 data set을 일단 shuffle한 다음에 처음부터 순서대로 split하기 때문에 **ex1-1**에서 발생한 문제를 방지할 수 있다.

* * *
다음으로 `stratify` parameter에 대해서 알아보자.  
**ex1-2**과 같이 shuffle을 한다고 하더라도 결국 shuffle 결과는 random이기 때문에,  
경우에 따라서는 shuffle을 했음에도 불구하고, positive(negative) class가 train set이나 test set 중 한 쪽으로 쏠릴 수가 있다.  
> 이 문제는 positive class와 negative class의 비율이 한 쪽이 클수록 더욱 발생 가능성이 높아진다.  
    >> 극단적으로 1 class가 48개, 0 class가 2개라고 한다면 train set(test set)에 0 class가 모두 몰리게 되는 경우가 매우 빈번하게 발생할 것이다.  

위 문제를 다음과 같이 해결할 수 있다.

In [15]:
# ex2. shuffle=True(default), stratify=ty
tX_train, tX_test, ty_train, ty_test = train_test_split(
    tX, ty, test_size=0.2, shuffle=True, stratify=ty, random_state=19)

In [16]:
print('==== ex1-2에서의 결과(stratify=None) ====')
print(30, 9)
print(5, 5)
print()
print('====   ex2에서의 결과(stratify=ty)   ====')
print(ty_train.count(1), ty_train.count(0))
print(ty_test.count(1), ty_test.count(0))

==== ex1-2에서의 결과(stratify=None) ====
30 9
5 5

====   ex2에서의 결과(stratify=ty)   ====
28 11
7 3


앞선 설명에서 다음과 같이 언급했었다.  
> The ratio between the positive class and the negative class is **$ 35 : 14 $**.

In [17]:
print('====      ty에서의 class ratio      ====')
print('35 / 14 = %f' % (35/14))
print()
print('==== ex1-2에서의 ratio(stratify=None) ====')
print('30 / 9 = %f' % (30/9), '--- tX_train')
print('5 / 5 = %f' % (5/5), '--- tX_test')
print()
print('====   ex2에서의 ratio(stratify=ty)   ====')
print('28 / 11 = %f' % (ty_train.count(1)/ty_train.count(0)), '--- tX_train')
print('7 / 3 = %f' % (ty_test.count(1)/ty_test.count(0)), '--- tX_test')

====      ty에서의 class ratio      ====
35 / 14 = 2.500000

==== ex1-2에서의 ratio(stratify=None) ====
30 / 9 = 3.333333 --- tX_train
5 / 5 = 1.000000 --- tX_test

====   ex2에서의 ratio(stratify=ty)   ====
28 / 11 = 2.545455 --- tX_train
7 / 3 = 2.333333 --- tX_test


(위 cell)  
`stratify=ty`로 지정하면 `ty`로부터 split되는 **`ty_train`과 `ty_test`에서의 1 class와 0 class의 비율**이 **`ty`에서의 1 class와 0 class의 비율**과 유사해지도록 split해준다.

# 1. K-NN Classification
Cross validation, Grid search를 모두 사용해보겠다(Random search의 경우, K-NN에서는 hyperparameter 경우의 수가 적다고 판단되어서 굳이 사용하지 않겠다).  

**summary:**  

| K-NN Classification | train_score | test(val)_score | n_neighbors |
| :---: | :---: | :---: | :---: |
| K-NN only | 0.863 | - | 3 |
| Cross validate | 0.859 | 0.733 | 3 |
| Grid search | 0.806 | 0.751 | 7 |

먼저, K-NN Classification의 경우 **features scale을 standardization**해주는 작업을 했을 때 유의미한 model performance enhanced를 기대할 수 있다.

In [26]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(X_train_scaled[:3])

[[-0.85135507 -0.98013068 -0.40478372 -0.55397256 -0.33131928 -0.60767846
   0.31079384 -0.79216928]
 [ 0.35657564  0.16144422  0.46536842  0.392787   -0.52639809 -0.30213902
  -0.11643851  0.56103382]
 [-0.5493724  -0.50447447 -0.62232176  1.21331196  0.14244354  0.3725939
  -0.76486207 -0.70759409]]


In [27]:
from sklearn.neighbors import KNeighborsClassifier

# default, n_neighbors=5
kn = KNeighborsClassifier(n_neighbors=3)

kn.fit(X_train_scaled, y_train)
kn.score(X_train_scaled, y_train)

0.8631921824104235

## case1. Cross validation only

In [28]:
from sklearn.model_selection import cross_validate
import numpy as np

# default, return_train_score=False, cv=5, n_jobs=1
# cv parameter is a value of k of k-fold
scores = cross_validate(kn, X_train_scaled, y_train,
                    return_train_score=True, cv=10, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8588522420525724 0.733051295610788


## case2. Grid search(Cross validation is included in Grid search)

In [29]:
from sklearn.model_selection import GridSearchCV

params = {'n_neighbors': [2, 3, 4, 5, 6, 7, 8], # default=5
          'weights': ['uniform', 'distance'],   # default='uniform'
          'p': [1, 2]                           # default=2
}

gs = GridSearchCV(estimator=kn, param_grid=params,
                  return_train_score=True, cv=5, n_jobs=-1)
gs.fit(X_train_scaled, y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(n_neighbors=3), n_jobs=-1,
             param_grid={'n_neighbors': [2, 3, 4, 5, 6, 7, 8], 'p': [1, 2],
                         'weights': ['uniform', 'distance']},
             return_train_score=True)

In [30]:
kn_grid_searched = gs.best_estimator_
print(kn_grid_searched)

KNeighborsClassifier(n_neighbors=7)


(위 cell)  
`weights`, `p` parameter는 default가 최적이라서 default로 지정되었기 때문에 출력되지 않는다.  

(다음 cell)  
* 이미 생성된 kn_grid_searched object의 attribute를 호출하여 확인할 수 있다.
* 또는 GridSearchCV object의 `best_params_` attribute를 호출하여 확인할 수 있다.

In [31]:
print(kn_grid_searched.weights, kn_grid_searched.p)
print(gs.best_params_)

uniform 2
{'n_neighbors': 7, 'p': 2, 'weights': 'uniform'}


In [32]:
print(kn_grid_searched.score(X_train_scaled, y_train))

0.8061889250814332


(위 cell)  
다음 code block을 보면 `n_neighbors=3`일 때가 `X_train_scaled`에 대한 score가 더 높은 것을 확인할 수 있다.
```
kn_grid_searched.n_neighbors = 3
kn_grid_searched.fit(X_train_scaled, y_train)
kn_grid_searched.score(X_train_scaled, y_train)
> 0.8631921824104235(is greater than 0.8061...)
```

처음에 내가 헷갈린 부분은 다음과 같다.

> 왜 `n_neighbors=3`일 때가 `X_train_scaled`에 대한 score가 더 높음에도 불구하고, `GridSearchCV`는 `n_neighbors`의 best value가 7이라고 하는가?

이에 대해서 조금 생각해보니 헷갈린 부분에 대해서 다음과 같이 정정할 수 있었다.  
* 결국 ML의 목표는 **실제 현장에 투입되어서 최선의 성과를 내는 것**이다.  
* training은 **Seen data**에 의해서 수행된다.  
* 하지만 **실제 현장에서 만나는 데이터**는 **Unseen data**이다.
    - 예제 실습 단계에서는 최종적으로 **test set이 Unseen data**에 해당하고, **cross validation 단계에서는 해당 단계에서의 test set(전체 입장에서 보면 validate set)**이 여기에 해당한다.
* 즉, **Unseen data에 대한 score가 높은 것**이 중요하다.  
    - Unseen data에 대한 score가 높은 model에 대해서 **[generalization ability가 뛰어나다](https://colab.research.google.com/drive/16M5guTPGlzBdq7E_fUua9BBF8LsE8p_j#scrollTo=Jd525wJ-peJ0)**고 한다.
    - 당연하게도 Unseen data에 대한 score만 높다고 좋은 것이 아니고, Seen data에 대한 score가 일정 수준으로 높고 Unseen data에 대한 score도 이에 유사한 수준으로 높은 상황을 말하는 것이다.
* 위와 같은 이유로 `GridSearchCV`는 **Unseen data 즉, validate set에 대한 score가 높도록 하는 parameter값**을 해당 parameter의 **optimal value**라고 설정(?)한다.
* 즉 **Seen data에 대한 score**는 0.86 : 0.80으로 `n_neighbors=3`일 때가 더 높을지 몰라도, **Unseen data에 대한 score**는 `n_neighbors=7`일 때가 더 높다는 것이다.  
* (다음 cell)  
자세한 결과는 다음 cell에서 확인할 수 있다.

In [33]:
max_value = np.max(gs.cv_results_['mean_test_score'])
print(f'>> max value of mean_test_score(정확히는 validate set score)\n{max_value} <<')
print()
# print(gs.cv_results_['mean_test_score'])
# print(gs.cv_results_['mean_train_score'])
for i in range(len(gs.cv_results_['mean_test_score'])):
    # train score가 1.0인 경우가 존재하는데, 터무니없으므로 제외함
    if gs.cv_results_['mean_train_score'][i] == 1.0: continue
    # 출력 결과를 보면, test score의 max가 train score의 max를 보장하지는 않음을 확인 가능
    # 출력 결과 기준으로 test score는 하단 3번 째가 최대이고, train score는 상단 4번 째가 최대
    print(gs.cv_results_['mean_test_score'][i], gs.cv_results_['mean_train_score'][i])
    print()

>> max value of mean_test_score(정확히는 validate set score)
0.7508596561375449 <<

0.6987471677995469 0.8346911065852003

0.6987471677995469 0.8407936350239267

0.7019858723177397 0.8469102379414833

0.7183126749300279 0.8591310251188051

0.7133946421431427 0.8053764509131852

0.7166333466613354 0.8135206066928287

0.7297081167532987 0.8151565578792244

0.7328935092629616 0.8167776066762705

0.7394508863121417 0.796421770735019

0.7296947887511662 0.8029365986124221

0.7476076236172198 0.8057837828887454

0.7508596561375449 0.8061911148643055

0.7394508863121418 0.7899061149470965

0.7508463281354125 0.7992722666534201



# 2. Logistic Regression
**summary:**  

| Logistic Regression | train_score | test(val)_score |
| :---: | :---: | :---: |
| default | 0.792 | 0.779 |
| Polynomial Features | 0.816 | 0.769 |
| Polynomial Features + Grid search | 0.795 | 0.785 |

In [44]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)

print(lr.predict(X_train_scaled[:50]))
print(lr.score(X_train_scaled, y_train))
print(lr.coef_, lr.intercept_)

[0 0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 0 1 0 0 0 0 1 0 1 0 0 0 1
 0 1 1 1 0 1 0 0 1 0 0 0 1]
0.7915309446254072
[[ 0.37299675  1.14382572 -0.19771633  0.06642437 -0.12729128  0.71380999
   0.25550662  0.18413458]] [-0.87494895]


(위 cell)  
이때 Logistic regression에서의 regression 자체의 predicted label value = $z$ 라고 했을 때 이 값을 sigmoid function에 대입하는데 이는 다음과 같다.  
$ sigmoid function = \sigma(z) = \frac{1}{1+e^{-z}} $  
그리고 기본적으로 다음과 같이 classify한다.  
> multiple-class classification에서는 softmax function을 사용한다.

* $ \sigma(z) > 0.5 $이면, positive class
* $ \sigma(z) <= 0.5 $이면, negative class  

그러면 다음과 같이 판단할 수 있겠다.  
* $ z = 0 $이면,  $ \sigma(z) = 0.5 $
* $ z \rightarrow \infty $이면, $ \sigma(z) \rightarrow 1.0 $
* _(질문) features 중에 coefficient가 높은 것이 중요한건지(부호 구분 yes), 아니면 coefficient의 absolute value가 높은 것이 중요한건지 궁금함._
    - 내 생각엔 후자가 맞는 것 같음.


## case1. cross validate only

(다음 cell)  
Linear regression을 사용한 경우 기본 model임에도 불구하고, performance(0.779)가 K-NN regression에서 hyperparameter tuning을 한 경우(0.751)보다도 우수함을 확인할 수 있다.

In [42]:
from sklearn.model_selection import cross_validate

scores = cross_validate(estimator=lr, X=X_train_scaled, y=y_train,
                        return_train_score=True, cv=5, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.7923484509794181 0.7785285885645742


> 또한 Linear regression을 사용한 classification에서는 cross validate만 사용한 case 이후에 Grid search, Random search 등으로 바로 넘어가지 않고, **Polynomial regression으로 변환**한 후에 model performance를 먼저 확인하고 결정하도록 하겠다.

## case2. convert to polynomial regression
처음 개념을 접할 때 polynomial과 multiple이 무슨 관계인지, 관계가 있는건지? 없는건지? 등이 헷갈렸다.  
이때 다음 문장들을 보면서 헷갈리는 부분을 해결할 수 있었기에 함께 첨부한다.

> Polynomial regression models a nonlinear relationship between the dependent variable and one or more independent variables(예제의 경우 당연하게도 more independent variables임 즉, **multiple polynomial regression**이라고 할 수 있겠음) by using polynomial terms.

> Polynomial regression includes high-degree terms of the features to capture nonlinear patterns.

> Polynomial regression is considerd linear because it is linear in the coefficients.

> Polynomial regression is used to capture non-linear relationships between the predictor(=label=dependent) and the responsible variables(=features=independent), but the model itself is considered as linear because it is linear in the parameters.

한국어로 의역해보자면 다음과 같이 할 수 있겠다.  
* Multiple은 feature의 개수와 관련된 개념이다. 본 예제에서는 $x_1$ ~ $x_8$ 8개의 features가 존재하므로 Multiple linear regression이다.
    - 1개의 feature만 존재하는 경우, simple linear regression이라고 한다.
* Polynomial은 feature의 차수와 관련된 개념이다. $x_1$은 1차, $(x_1)^2$은 2차.
    - function 내에서 가장 큰 차수가 제곱 이상이라면 polynomial이고, 모두 1차항이라면 linear이다.
    - 이 부분이 조금 헷갈릴 수 있는데, $y = ax_1^2 + bx_1 + c $는 가장 큰 차수가 제곱이기 때문에 polynomial이지만, $x_1^2$ 자체가 $ x_1 $으로부터 파생된 것이기 때문에 linear로 취급한다.
        + 이때 $x_1$을 original feature, $x_1^2$을 derived feature라고 한다.
        + 위 영어 문장 중에서 __Polynomial regression is considerd linear__ 부분을 참고

In [54]:
from sklearn.preprocessing import PolynomialFeatures

# default, degree=2, interaction_only=False, include_bias=True
poly = PolynomialFeatures(degree=2, interaction_only=False, include_bias=False)
poly.fit(X_train_scaled)
X_train_poly = poly.transform(X_train_scaled)

# features가 8개 -> 44개로 늘어난 것을 확인할 수 있다
print(X_train_scaled.shape, X_train_poly.shape)

(614, 8) (614, 44)


> 이때 `PolynomialFeatures`를 통해서 features를 추가해줬기 때문에, 새롭게 fitting이 필요하다는 것을 주의하자.

In [55]:
lr.fit(X_train_poly, y_train)

LogisticRegression()

In [56]:
scores = cross_validate(estimator=lr, X=X_train_poly, y=y_train,
                        return_train_score=True, cv=5, n_jobs=-1)

print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8163727584322686 0.7688258030121286


(위 cell)  
`PolynomialFeatures` class를 사용해서 feature의 개수를 8개에서 44개로 늘렸다.  
이때 오히려 model performance가 감소한 것을 확인할 수 있는데, feature의 수가 너무 많아졌기 때문이라고 추정할 수 있겠다.  
따라서 `LinearRegression`의 `C` parameter를 조정해서 model complexity를 낮춰보겠다.  
> 조정하는 김에 아예 **case3**으로 해서 hyperparameter tuning을 하겠다.

## case3. Hyperparameter tuning by GridSearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV

param_grid = {'max_iter': [100, 500, 1000],
              'solver': ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga'],
              'C': [0.001, 0.01, 0.1, 1.0, 5.0, 10.0],
              'penalty': ['l1', 'l2']
              }

grid_search = GridSearchCV(estimator=LogisticRegression(), param_grid=param_grid,
                           return_train_score=True, n_jobs=-1)

grid_search.fit(X_train_poly, y_train)

In [59]:
grid_search.best_params_

{'C': 0.1, 'max_iter': 100, 'penalty': 'l1', 'solver': 'liblinear'}

In [62]:
print(np.max(grid_search.cv_results_['mean_test_score']))
print(grid_search.cv_results_['mean_test_score'])

nan
[       nan        nan 0.65147274        nan 0.65147274 0.67756897
 0.67756897 0.7442623  0.67756897 0.680821          nan        nan
 0.65147274        nan 0.65147274 0.67756897 0.67756897 0.7442623
 0.67756897 0.67756897        nan        nan 0.65147274        nan
 0.65147274 0.67756897 0.67756897 0.7442623  0.67756897 0.67756897
        nan        nan 0.70371851        nan 0.65147274 0.75902972
 0.75902972 0.77366387 0.76229508 0.76066907        nan        nan
 0.70371851        nan 0.65147274 0.75902972 0.75902972 0.77366387
 0.75901639 0.76064241        nan        nan 0.70371851        nan
 0.65147274 0.75902972 0.75902972 0.77366387 0.75901639 0.76064241
        nan        nan 0.78505931        nan 0.78019459 0.77046515
 0.77046515 0.77206451 0.77045182 0.76881248        nan        nan
 0.78505931        nan 0.77369052 0.77046515 0.77046515 0.77206451
 0.77046515 0.76883913        nan        nan 0.78505931        nan
 0.77369052 0.77046515 0.77046515 0.77206451 0.77046515 0.7

In [65]:
tmp_score = 0.0
tmp_index = 0
for i in range(len(grid_search.cv_results_['mean_test_score'])):
    if grid_search.cv_results_['mean_test_score'][i] == 'nan':
        continue
    if grid_search.cv_results_['mean_test_score'][i] >= tmp_score:
        tmp_score = grid_search.cv_results_['mean_test_score'][i]
        tmp_index=i

In [66]:
print(tmp_score)
print(tmp_index)

0.7850593096094897
82


In [68]:
print(grid_search.cv_results_['mean_train_score'][tmp_index])

0.7947916149222591


(위 cell, 그 위 cell)  
유의미한 model performance enhanced를 이뤘다고 볼 수 있겠다.  

* `PolynomialFeatures` 사용 전: 0.779  
* `PolynomialFeatures` 사용 후: 0.769  
* `GridSearchCV` &nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;사용 후: 0.785